In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os, glob, random, matplotlib, cv2
import matplotlib.patches as mplpatches
from collections import Counter
import pandas as pd
from xml.etree import ElementTree as ET
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers

In [ ]:
!pip install xmltodict

In [ ]:
import xmltodict

In [ ]:
annotations_path = "/kaggle/input/face-mask-detection/annotations"
images_path = "/kaggle/input/face-mask-detection/images"

CHANNELS = 3
IMAGE_SIZE = 224
INPUT_SIZE = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
TRAIN_SET_PRCNT = 0.85

In [ ]:
def get_annotation(annotation_file):

    objects = {
            "xmin":[],
            "ymin":[],
            "xmax":[],
            "ymax":[],
            "name":[],
            "file":[],
            "width":[],
            "height":[],
           }

    tree = ET.parse(annotation_file)

    for el in tree.iter():
        if 'size' in el.tag:
            for attr in list(el):
                if 'width' in attr.tag:
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))

        if 'object' in el.tag:
            for attr in list(el):

                if 'name' in attr.tag:
                    name = attr.text
                    objects['name']+=[name]
                    objects['width']+=[width]
                    objects['height']+=[height]
                    objects['file']+=[annotation_file.split('/')[-1][0:-4]]

                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            objects['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            objects['ymin']+=[ymin]
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            objects['xmax']+=[xmax]
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            objects['ymax']+=[ymax]

    return objects


In [ ]:
# list of paths to images and annotations
image_files = [
    f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f))
]
annot_files = [
    f for f in os.listdir(annotations_path) if os.path.isfile(os.path.join(annotations_path, f))
]

image_files.sort()
annot_files.sort()

images, targets = [], []

# loop over the annotations and images, preprocess them and store in lists
for i in range(0, len(annot_files)):

    # Access bounding box coordinates
    annot = get_annotation(os.path.join(annotations_path, annot_files[i]))

    top_left_x, top_left_y = annot['xmax'], annot['ymax']
    bottom_right_x, bottom_right_y = annot['xmin'], annot['ymin']

    image = keras.utils.load_img(
        os.path.join(images_path, image_files[i]),
    )
    (w, h) = image.size[:2]

    # resize train set images
    #if i < int(len(annot_files) * TRAIN_SET_PRCNT):
        # resize image if it is for training dataset
    #    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    # convert image to array and append to list
    #images.append(keras.utils.img_to_array(image))
    #targets.append((annot['xmax'], annot['ymax'],annot['xmin'], annot['ymin']))

    # apply relative scaling to bounding boxes as per given image and append to list
    for i in range(0, len(top_left_x)):
        # convert image to array and append to list
        images.append(keras.utils.img_to_array(image))
        targets.append(
            (
                float(top_left_x[i]) / w,
                float(top_left_y[i]) / h,
                float(bottom_right_x[i]) / w,
                float(bottom_right_y[i]) / h,
            )
        )



# Convert the list to numpy array, split to train and test dataset
(x_train), (y_train) = (
    np.asarray(images[: int(len(images) * TRAIN_SET_PRCNT)]),
    np.asarray(targets[: int(len(targets) * TRAIN_SET_PRCNT)]),
)

(x_test), (y_test) = (
    np.asarray(images[int(len(images) * TRAIN_SET_PRCNT) :]),
    np.asarray(targets[int(len(targets) * TRAIN_SET_PRCNT) :]),
)

In [ ]:
input_image = x_train[0]
fig, ax1 = plt.subplots(1, figsize=(15, 15))
im = input_image
# Display the image
ax1.imshow(im.astype("uint8"))


input_image = cv2.resize(
    input_image, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA
)
#input_image = np.expand_dims(input_image, axis=0)
#preds = vit_object_detector.predict(input_image)[0]
#
(h, w) = (im).shape[0:2]
#
for i in range(0,3):
    top_left_x, top_left_y = int(y_train[i][0] * 224), int(y_train[i][1] * 224)
    #
    bottom_right_x, bottom_right_y = int(y_train[i][2] * 224), int(y_train[i][3] * 224)
    #
    box_predicted = [top_left_x, top_left_y, bottom_right_x, bottom_right_y]
    #
    ## Create the bounding box
    rect = mplpatches.Rectangle(
                            (top_left_x, top_left_y),
                            bottom_right_x - top_left_x,
                            bottom_right_y - top_left_y,
                            facecolor="none",
                            edgecolor="red",
                            linewidth=1,
                        )

    ax1.add_patch(rect)

plt.show()

In [ ]:
# Count how many files there are in annotation path. It should be the same number as in images path
count = 0
# Iterate directory
for path in os.listdir(annotations_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(annotations_path, path)):
        count += 1
print('File count:', count)

In [ ]:
# Let's find out how many people are wearing mask, not wearing it or wearing it wrong:
img_names = []
xml_names = []
for dirname, _, filenames in os.walk(images_path):
    for filename in filenames:
        img_names.append(filename)

img_annot_list = []
for i in img_names[:]:
    with open(os.path.join(annotations_path,i[:-4]+".xml")) as fd:
        doc = xmltodict.parse(fd.read())
    temp = doc["annotation"]["object"]
    if type(temp) == list:
        for i in range(len(temp)):
            img_annot_list.append(temp[i]["name"])
    else:
        img_annot_list.append(temp["name"])


classes = Counter(img_annot_list).keys()
target = Counter(img_annot_list).values()
print(list(zip(classes,target)))

In [ ]:
matplotlib.pyplot.pie(target,
                      labels=classes,
                      colors=['green', 'yellow', 'red'],
                      pctdistance=0.8,
                      shadow=True,
                      labeldistance=1.1,
                      startangle=75,
                      radius=2,
                      counterclock=True)

In [ ]:
def get_df_annotations():
    objects = {
            "xmin":[],
            "ymin":[],
            "xmax":[],
            "ymax":[],
            "name":[],
            "file":[],
            "width":[],
            "height":[],
           }

    for an_path in glob.glob(annotations_path+"/*.xml"):
        tree = ET.parse(an_path)

        for el in tree.iter():
            if 'size' in el.tag:
                for attr in list(el):
                    if 'width' in attr.tag:
                        width = int(round(float(attr.text)))
                    if 'height' in attr.tag:
                        height = int(round(float(attr.text)))

            if 'object' in el.tag:
                for attr in list(el):

                    if 'name' in attr.tag:
                        name = attr.text
                        objects['name']+=[name]
                        objects['width']+=[width]
                        objects['height']+=[height]
                        objects['file']+=[an_path.split('/')[-1][0:-4]]

                    if 'bndbox' in attr.tag:
                        for dim in list(attr):
                            if 'xmin' in dim.tag:
                                xmin = int(round(float(dim.text)))
                                objects['xmin']+=[xmin]
                            if 'ymin' in dim.tag:
                                ymin = int(round(float(dim.text)))
                                objects['ymin']+=[ymin]
                            if 'xmax' in dim.tag:
                                xmax = int(round(float(dim.text)))
                                objects['xmax']+=[xmax]
                            if 'ymax' in dim.tag:
                                ymax = int(round(float(dim.text)))
                                objects['ymax']+=[ymax]

    return objects

objects = get_df_annotations()

In [ ]:
df = pd.DataFrame(objects)
df.head(10)

In [ ]:
file_name_str = 'maksssksksss737'
filterr = df[df['file'] == file_name_str]
# path
path = fr'/kaggle/input/face-mask-detection/images/{file_name_str}.png'

# Reading an image in default mode
image = cv2.imread(path)
# Window name in which image is displayed
window_name = 'Image'

# Blue color in BGR
color = (255, 0, 0)

# Line thickness of 2 px
thickness = 2

# Using cv2.rectangle() method
# Draw a rectangle with blue line borders of thickness of 2 px
cv2.rectangle(image, (46, 71), (28, 55), color, thickness) #(xmax,ymax), (xmin,ymin)
cv2.rectangle(image, (111, 78), (98, 62), color, thickness)
cv2.rectangle(image, (193, 90), (159, 50), color, thickness)
cv2.rectangle(image, (313, 80), (293, 59), color, thickness)
cv2.rectangle(image, (372, 72), (352, 52), color, thickness)
cv2.rectangle(image, (241, 73), (228, 53), color, thickness)

# Displaying the image
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
df.info()

In [ ]:
result = list(zip(df.width, df.height))
cntr = dict(Counter(result).most_common(10))
df_aux = pd.DataFrame.from_dict(Counter(cntr), orient='index', columns=['Width_height'])
df_aux.plot.bar()
plt.ylabel('Counts per width-height pair')
plt.xlabel('Width-height pair')
plt.title('Are the images the same size?')

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
SEED = 42
DROPOUT = 0.1
NUM_HEADS = 8
ACTIVATION = "relu" # Activation function
PADDING = "VALID"
EPSILON = 1e-6
EPOCHS = 30
PATCH_SIZE = 32 # Size of the patches to be extracted from the input images
NUM_LAYERS = 8
WEIGHT_DECAY = 1e-4
BATCH_SIZE = 4
MLP_DIM = 128
PROJECTION_DIM = 64
LR = 1e-3

In [ ]:
# Implement multilayer-perceptron (MLP)
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    #     Override function to avoid error while saving model
    def get_config(self):
        config = super().get_config().copy()
        config.update(
            {
                "input_shape": input_shape,
                "patch_size": patch_size,
                "num_patches": num_patches,
                "projection_dim": projection_dim,
                "num_heads": num_heads,
                "transformer_units": transformer_units,
                "transformer_layers": transformer_layers,
                "mlp_head_units": mlp_head_units,
            }
        )
        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding=PADDING,
        )
        # return patches
        return tf.reshape(patches, [batch_size, -1, patches.shape[-1]])


In [ ]:
### Display patches for an input image

random_num = random.randint(0, len(x_train)-1)

plt.figure(figsize=(4, 4))
plt.imshow(x_train[random_num]/255.)
plt.axis("off")

patches = Patches(PATCH_SIZE)(tf.convert_to_tensor([x_train[random_num]]))
print(f"Image size: {IMAGE_SIZE} X {IMAGE_SIZE}")
print(f"Patch size: {PATCH_SIZE} X {PATCH_SIZE}")
print(f"{patches.shape[1]} patches per image \n{patches.shape[-1]} elements per patch")


n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(6, 6))

for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (PATCH_SIZE, PATCH_SIZE, CHANNELS))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")


In [ ]:
class PatchEncoder(layers.Layer):
    # Implement the patch encoding layer

    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    # Override function to avoid error while saving model
    def get_config(self):
        config = super().get_config().copy()
        config.update(
            {
                "input_shape": input_shape,
                "patch_size": patch_size,
                "num_patches": num_patches,
                "projection_dim": projection_dim,
                "num_heads": num_heads,
                "transformer_units": transformer_units,
                "transformer_layers": transformer_layers,
                "mlp_head_units": mlp_head_units,
            }
        )
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


In [ ]:
def create_vit_object_detector(
                                input_shape,
                                patch_size,
                                num_patches,
                                projection_dim,
                                num_heads,
                                transformer_units,
                                transformer_layers,
                                mlp_head_units):


    inputs = layers.Input(shape=input_shape)
    # Create patches
    patches = Patches(PATCH_SIZE)(inputs)
    # Encode patches
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=EPSILON)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=DROPOUT
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=EPSILON)(x2)
        # MLP
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=DROPOUT)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=EPSILON)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(DROPOUT*3)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=DROPOUT*3)

    bounding_box = layers.Dense(4)(
                                    features
                                  )  # Final four neurons that output bounding box

    # return Keras model.
    return keras.Model(inputs=inputs, outputs=bounding_box)


In [ ]:
from tensorflow.python.client import device_lib
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')), "\n")

print(device_lib.list_local_devices())

tf.debugging.set_log_device_placement(True)
tf.random.set_seed(SEED)

In [ ]:
def run_experiment(model, learning_rate, weight_decay, batch_size, num_epochs):

    optimizer = tfa.optimizers.AdamW(
                                        learning_rate=learning_rate,
                                        weight_decay=weight_decay
                                    )

    # Compile model.
    model.compile(
                    optimizer=optimizer,
                    loss=[keras.losses.MeanSquaredError(), tfa.losses.GIoULoss()],
                    #metrics=[tf.keras.metrics.MeanIoU(num_classes=len(classes)+1)]

                  )

    checkpoint_filepath = "logs/"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
                            x=x_train,
                            y=y_train,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            validation_split=0.25,
                            callbacks=[
                                        checkpoint_callback,
                                        keras.callbacks.EarlyStopping(monitor="val_loss", patience=10),
                                      ],
                         )

    return history


num_patches = (IMAGE_SIZE // PATCH_SIZE) ** 2


# Size of the transformer layers
transformer_units = [
                        PROJECTION_DIM * 2,
                        PROJECTION_DIM,
                    ]

transformer_layers = 6
mlp_head_units = [2048, 1024, 512, 64, 32]  # Size of the dense layers


history = []
num_patches = (IMAGE_SIZE // PATCH_SIZE) ** 2

vit_object_detector = create_vit_object_detector(
                                                    INPUT_SIZE,
                                                    PATCH_SIZE,
                                                    num_patches,
                                                    PROJECTION_DIM,
                                                    NUM_HEADS,
                                                    transformer_units,
                                                    transformer_layers,
                                                    mlp_head_units,
                                                )

In [ ]:
vit_object_detector.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(vit_object_detector, to_file='model.png')

In [ ]:
# Train model
history = run_experiment(
                        vit_object_detector, LR, WEIGHT_DECAY, BATCH_SIZE, EPOCHS
                        )